In [1]:
# Importing libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
# Importing MicroBiologyEvents Table and dropping empty rows in ORG_NAME AND AB_NAME
MBE_DF = pd.read_csv('MICROBIOLOGYEVENTS.csv', delimiter=',')
MBE_DF.dropna(subset=['ORG_NAME','AB_NAME'], how='any', inplace=True)
MBE_DF.drop_duplicates(inplace=True)

In [3]:
# Dropping Inessential columns from the dataframe
MBE_DF = MBE_DF.drop(['ROW_ID', 'DILUTION_TEXT','DILUTION_COMPARISON','DILUTION_VALUE','ORG_ITEMID','ISOLATE_NUM','AB_ITEMID', 'CHARTDATE','SPEC_ITEMID'],
                     axis = 1)

In [4]:
# Switching Intermediate interpretation to Resistant
MBE_DF = MBE_DF[MBE_DF.INTERPRETATION != 'P']
MBE_DF.loc[MBE_DF.INTERPRETATION == 'I','INTERPRETATION'] = 'R'

In [5]:
#Importing Demographic table and dropping inessential columns from the dataframe
DG_DF = pd.read_csv('DEMOGRAPHICS.csv', delimiter=',')
DG_DF=DG_DF.drop(['dod', 'icustay_id', 'dischtime', 'los_hospital', 'ethnicity', 'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'intime', 'outtime', 'los_icu', 'icustay_seq', 'first_icu_stay'], axis = 1)
DG_DF.drop_duplicates(inplace=True)

In [6]:
# Cleaning age column, age 300 in the dataset  is originally equal to 90
def clean_age(age):
  if age == 300:
    return 90
  elif age > 300:
    dif = age - 300
    age = 90+dif
    return age
  elif age == 0:
    return 54
  return age
DG_DF['admission_age'] = DG_DF['admission_age'].apply(clean_age)

In [7]:
#Merging Demographics and MicroBiologyEvents table on SUBJECT_ID','HADM_ID
merged_df = MBE_DF.merge(DG_DF,left_on=['SUBJECT_ID','HADM_ID'] ,right_on = ['subject_id','hadm_id']).drop_duplicates()
merged_df= merged_df.drop(['subject_id','hadm_id'],axis=1)
merged_df.dropna(inplace=True)

In [8]:
#Reading Diagnoses table and dropping inessential columns
D_ICD_DF = pd.read_csv('DIAGNOSES_ICD.csv', delimiter=',').dropna()
D_ICD_DF = D_ICD_DF.drop(['ROW_ID', 'SEQ_NUM'], axis=1)

In [9]:
#Dropping all the ICD9_CODES which are starting with'E'
test_DF = D_ICD_DF[~D_ICD_DF['ICD9_CODE'].str.startswith('E')]

In [10]:
# Taking first three digits of the ICD9 Code and replacing the code with Diagnoses string
def icd_codes(code):
    if 'V' in code:
        code = int(code[1:3])
        if code >= 1  and code <= 91:
            return 'Supplementary Classification Of Factors Influencing Health Status And Contact With Health Services'
    else:    
        code = int(code[0:3]) 
        if code >= 1 and code <= 139:
            return 'Infectious And Parasitic Diseases'
        elif code >= 140 and code <= 239:
            return 'Neoplasms'
        elif code >= 240 and code <= 279:
            return 'Endocrine, Nutritional And Metabolic Diseases, And Immunity Disorders'
        elif code >= 280 and code <= 289:
            return 'Diseases Of The Blood And Blood-Forming Organs'
        elif code == 290 and code <= 319:
            return 'Mental Disorders'
        elif code >= 320 and code <= 389:
            return 'Diseases Of The Nervous System And Sense Organs'
        elif code >= 390 and code <= 459:
            return 'Diseases Of The Circulatory System'
        elif code >= 460 and code <= 519:
            return 'Diseases Of The Respiratory System'
        elif code >= 520 and code <= 579:
            return 'Diseases Of The Digestive System'
        elif code >= 580 and code <= 629:
            return 'Diseases Of The Genitourinary System'
        elif code >= 630 and code <= 679:
            return 'Complications Of Pregnancy, Childbirth, And The Puerperium'
        elif code >= 680 and code <= 709:
            return 'Diseases Of The Skin And Subcutaneous Tissue'
        elif code >= 710 and code <= 739:
            return 'Diseases Of The Musculoskeletal System And Connective Tissue'
        elif code >= 740 and code <= 759:
            return 'Congenital Anomalies'
        elif code >= 760 and code <= 779:
            return 'Certain Conditions Originating In The Perinatal Period'
        elif code >= 780 and code <= 799:
            return 'Symptoms, Signs, And Ill-Defined Conditions'
        elif code >= 800 and code <= 999:
            return 'Injury And Poisoning'
    return '0'

In [11]:
#Applying the function to the dataframe
test_DF['ICD9_CODE'] = test_DF['ICD9_CODE'].apply(icd_codes)

<ipython-input-11-94091ad45610>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_DF['ICD9_CODE'] = test_DF['ICD9_CODE'].apply(icd_codes)


In [14]:
#Merging Diagnoses dataframe with Demographics and MicroBiologyEvents
diag_df = merged_df.merge(new_df,on=['SUBJECT_ID','HADM_ID']).drop_duplicates().dropna()

In [16]:
#Calculating collection_interval in days
diag_df['admittime'] = pd.to_datetime(diag_df['admittime'])
diag_df['CHARTTIME'] = pd.to_datetime(diag_df['CHARTTIME'])
diag_df['collection_interval'] = diag_df['CHARTTIME'] - diag_df['admittime']
diag_df['collection_interval'] = diag_df['collection_interval'] / (np.timedelta64(1,'h') * 24)
diag_df = diag_df.round(1)

In [17]:
# Not taking into account negative collection intervals
MicroBiologyDF = diag_df[diag_df['collection_interval']>0].drop(['CHARTTIME','admittime'],axis=1)

In [18]:
#Reading admissions table and calculating the number of admissions for each patient
ADM_DF = pd.read_csv('ADMISSIONS.csv', delimiter=',')
p_adm= ADM_DF['SUBJECT_ID'].value_counts()
p_adm_df= p_adm.to_frame().reset_index()
p_adm_df = p_adm_df.rename(columns={'index':'SUBJECT_ID','SUBJECT_ID':'previous_admissions'})